In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [2]:
df_train = pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')

In [3]:
df_train.drop('Survived',1,inplace=True)

In [4]:
combined = df_train.append(df_test)
combined.reset_index(inplace=True)
combined.drop('index',inplace=True,axis=1)

In [5]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


In [6]:
combined.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
len(combined.Ticket.unique())

929

In [8]:
pattern = re.compile('([A-Za-z]+)\.')
def getTitle(x):
    result = pattern.search(x)
    if result:
        return result.group(1).strip()
    else:
        return ''
combined['Title'] = combined['Name'].map(getTitle)

In [9]:
combined['Title'] = combined['Title'].replace(['Capt', 'Col','Major','Dr', 'Rev'], 'Officer')
combined['Title'] = combined['Title'].replace(['Don', 'Sir', 'Jonkheer', 'Dona','Countess','Lady'], 'Royalty')

combined['Title'] = combined['Title'].replace(['Ms','Mme'], 'Mrs')
combined['Title'] = combined['Title'].replace(['Mlle'], 'Miss')


In [10]:
combined.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Royalty', 'Officer'], dtype=object)

In [11]:
grouped = combined.groupby(['Sex','Pclass','Title'])
grouped.median()

PassengerId   Age  SibSp  Parch      Fare
Sex    Pclass Title                                             
female 1      Miss           529.5  30.0    0.0    0.0   99.9625
              Mrs            853.5  45.0    1.0    0.0   78.1125
              Officer        797.0  49.0    0.0    0.0   25.9292
              Royalty        760.0  39.0    0.0    0.0   86.5000
       2      Miss           606.5  20.0    0.0    0.0   20.2500
              Mrs            533.0  30.0    1.0    0.0   26.0000
       3      Miss           603.5  18.0    0.0    0.0    8.0500
              Mrs            668.5  31.0    1.0    1.0   15.5000
male   1      Master         803.0   6.0    1.0    2.0  134.5000
              Mr             634.0  41.5    0.0    0.0   47.1000
              Officer        678.0  52.0    0.0    0.0   37.5500
              Royalty        600.0  40.0    0.0    0.0   27.7208
       2      Master         550.0   2.0    1.0    1.0   26.0000
              Mr             723.5  30.0    0.0    0.0   13.0000
              Officer        513.0  41.5    0.0    0.0   13.0000
       3      Master         789.0   6.0    3.0    1.0   22.3583
              Mr             640.5  26.0    0.0    0.0    7.8958

In [12]:
def process_age():
    
    global combined
    
    # a function that fills the missing values of the Age variable
    
    def fillAges(row):
        if row['Sex']=='female' and row['Pclass'] == 1:
            if row['Title'] == 'Miss':
                return 30
            elif row['Title'] == 'Mrs':
                return 45
            elif row['Title'] == 'Officer':
                return 49
            elif row['Title'] == 'Royalty':
                return 39

        elif row['Sex']=='female' and row['Pclass'] == 2:
            if row['Title'] == 'Miss':
                return 20
            elif row['Title'] == 'Mrs':
                return 30

        elif row['Sex']=='female' and row['Pclass'] == 3:
            if row['Title'] == 'Miss':
                return 18
            elif row['Title'] == 'Mrs':
                return 31

        elif row['Sex']=='male' and row['Pclass'] == 1:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 41.5
            elif row['Title'] == 'Officer':
                return 52
            elif row['Title'] == 'Royalty':
                return 40

        elif row['Sex']=='male' and row['Pclass'] == 2:
            if row['Title'] == 'Master':
                return 2
            elif row['Title'] == 'Mr':
                return 30
            elif row['Title'] == 'Officer':
                return 41.5

        elif row['Sex']=='male' and row['Pclass'] == 3:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 26
    
    combined.Age = combined.apply(lambda r : fillAges(r) if np.isnan(r['Age']) else r['Age'], axis=1)

In [13]:
process_age()

In [14]:
Pclass_dummies_combined=pd.get_dummies(combined.Pclass,prefix="Pclass",drop_first=True)
combined=pd.merge(combined,Pclass_dummies_combined,left_index=True, right_index=True)

In [15]:
combined['Embarked'] = combined['Embarked'].fillna('C')

In [16]:
Embarked_dummies_combined=pd.get_dummies(combined.Embarked,prefix="Embarked", drop_first=True)
combined=pd.merge(combined,Embarked_dummies_combined,left_index=True, right_index=True)

In [17]:
combined['Sex'] = combined['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [18]:
combined['FamilySize'] = combined['SibSp'] + combined['Parch'] + 1

In [19]:
combined['IsAlone'] = 0
combined.loc[(1<combined['FamilySize']) &(combined['FamilySize'] <5), 'IsAlone'] = 2
combined.loc[combined['FamilySize'] >=5, 'IsAlone'] = 1

In [20]:
IsAlone_dummies_combined=pd.get_dummies(combined.IsAlone,prefix="IsAlone", drop_first=True)
combined=pd.merge(combined,IsAlone_dummies_combined,left_index=True, right_index=True)

In [21]:
combined.drop(['SibSp','Parch','FamilySize','IsAlone'],inplace=True,axis=1)

In [22]:
combined['Fare'] = combined['Fare'].fillna(combined['Fare'].median())

In [23]:
# replacing missing cabins with U (for Uknown)
combined.Cabin.fillna('U',inplace=True)
    
    # mapping each Cabin value with the cabin letter
combined['Cabin'] = combined['Cabin'].map(lambda c : c[0])
    
    # dummy encoding ...
cabin_dummies = pd.get_dummies(combined['Cabin'],prefix='Cabin', drop_first=True)
    
combined = pd.concat([combined,cabin_dummies],axis=1)
    
combined.drop('Cabin',axis=1,inplace=True)

In [24]:
def cleanTicket(ticket):
        ticket = ticket.replace('.','')
        ticket = ticket.replace('/','')
        ticket = ticket.split()
        ticket = map(lambda t : t.strip() , ticket)
        ticket = filter(lambda t : not t.isdigit(), ticket)
        if len(ticket) > 0:
            return ticket[0]
        else: 
            return 'XXX'
    

    # Extracting dummy variables from tickets:

combined['Ticket'] = combined['Ticket'].map(cleanTicket)
tickets_dummies = pd.get_dummies(combined['Ticket'],prefix='Ticket')
combined = pd.concat([combined, tickets_dummies],axis=1)
combined.drop('Ticket',inplace=True,axis=1)

In [25]:
combined.drop('Name',axis=1,inplace=True)

In [26]:
Title_dummies_combined=pd.get_dummies(combined.Title,prefix="Title",drop_first=True)
combined=pd.merge(combined,Title_dummies_combined,left_index=True, right_index=True)
combined.drop('Title',axis=1,inplace=True)

In [27]:
combined.drop(['Pclass','Embarked'],axis=1,inplace=True)

In [28]:
features = list(combined.columns)
features.remove('PassengerId')
combined[features] = combined[features].apply(lambda x: x/x.max(), axis=0)

In [29]:
combined.head()

,PassengerId,Sex,Age,Fare,Pclass_2,Pclass_3,Embarked_Q,Embarked_S,IsAlone_1,IsAlone_2,...,Ticket_STONOQ,Ticket_SWPP,Ticket_WC,Ticket_WEP,Ticket_XXX,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,1,1.0,0.2750,0.014151,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,0.0,0.4750,0.139136,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,0.0,0.3250,0.015469,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,0.0,0.4375,0.103644,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,5,1.0,0.4375,0.015713,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [30]:
def recover_train_test_target():
    global combined
    
    train0 = pd.read_csv('train.csv')
    
    y_train = train0.Survived
    X_train = combined.ix[0:890]
    X_test = combined.ix[891:]
    
    return y_train,X_train,X_test

In [31]:
y_train,X_train,X_test=recover_train_test_target()

In [32]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.cross_validation import StratifiedKFold
clf = ExtraTreesClassifier(n_estimators=200)
clf = clf.fit(X_train, y_train)

In [33]:
features = pd.DataFrame()
features['features'] = X_train.columns
features['importance'] = clf.feature_importances_

In [34]:
features.sort(['importance'],ascending=False)

C:\Users\phong\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,features,importance
0,PassengerId,0.140148
2,Age,0.136693
3,Fare,0.125713
56,Title_Mr,0.124740
1,Sex,0.117518
5,Pclass_3,0.045162
57,Title_Mrs,0.035376
17,Cabin_U,0.034330
55,Title_Miss,0.033898
9,IsAlone_2,0.031616


In [35]:
model = SelectFromModel(clf, prefit=True)
X_train_new = model.transform(X_train)
X_train_new.shape

(891L, 12L)

In [36]:
X_test_new = model.transform(X_test)
X_test_new.shape

(418L, 12L)

In [37]:
from sklearn import tree
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.cluster import KMeans
from sklearn import decomposition
from sklearn import cross_validation, metrics   
from sklearn.grid_search import GridSearchCV

In [38]:
cross_validation = StratifiedKFold(y_train, n_folds=5)

In [39]:
from sklearn.cross_validation import train_test_split
X_train_divided, X_test_divided,y_train_divided,y_test_divided = train_test_split(X_train_new, y_train, test_size=0.3, random_state=1)

In [40]:
clf_rdf = RandomForestClassifier()
clf_rdf=clf_rdf.fit(X_train_divided,y_train_divided)
clf_rdf.score(X_test_divided,y_test_divided)

0.75

In [41]:
clf_svm = svm.SVC(probability=True)
clf_svm=clf_svm.fit(X_train_divided,y_train_divided)
clf_svm.score(X_test_divided,y_test_divided)

0.63059701492537312

In [42]:
clf_dt = tree.DecisionTreeClassifier()
clf_dt=clf_dt.fit(X_train_divided,y_train_divided)
clf_dt.score(X_test_divided,y_test_divided)

0.75

In [43]:
clf_lg=LogisticRegression()
clf_lg=clf_lg.fit(X_train_divided, y_train_divided)
clf_lg.score(X_test_divided,y_test_divided)

0.78731343283582089

In [44]:
clf_gb=GradientBoostingClassifier()
clf_gb=clf_gb.fit(X_train_divided,y_train_divided)
clf_gb.score(X_test_divided,y_test_divided)

0.77985074626865669

In [45]:
rdf_parameter_grid = {
                 'max_depth' : [4,5,6,7,8],
                 'n_estimators': [200,210,240,250,300],
                 'criterion': ['gini','entropy']
                 }
rdf_grid_search = GridSearchCV(clf_rdf,
                           param_grid=rdf_parameter_grid,
                           cv= cross_validation,n_jobs=-1)
rdf_grid_search.fit(X_train_new,y_train)

print('Best score: {}'.format(rdf_grid_search.best_score_))
print('Best parameters: {}'.format(rdf_grid_search.best_params_))

Best score: 0.828282828283
Best parameters: {'n_estimators': 200, 'criterion': 'gini', 'max_depth': 4}


In [46]:
gb_parameter_grid = {
                'max_features': ['sqrt', 'log2'],
                 'max_depth' : [4,5,6,7,8],
                 'n_estimators': [200,210,240,250,300],
                'learning_rate': [0.01,0.05,0.1,0.2,0.3,0.4]
                 }
gb_grid_search = GridSearchCV(clf_gb,
                           param_grid=gb_parameter_grid,
                           cv= cross_validation,n_jobs=-1)
gb_grid_search.fit(X_train_new,y_train)

print('Best score: {}'.format(gb_grid_search.best_score_))
print('Best parameters: {}'.format(gb_grid_search.best_params_))

Best score: 0.821548821549
Best parameters: {'max_features': 'sqrt', 'n_estimators': 210, 'learning_rate': 0.4, 'max_depth': 5}


In [47]:
rdf_grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [52]:
gb_grid_search.best_estimator_

GradientBoostingClassifier(init=None, learning_rate=0.4, loss='deviance',
              max_depth=5, max_features='sqrt', max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=210,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [53]:
Best_param_predict=gb_grid_search.best_estimator_.fit(X_train_new,y_train)

In [54]:
predict=Best_param_predict.predict(X_test_new)

In [55]:
test= pd.read_csv('test.csv')

In [56]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": predict
    })
submission.to_csv('titanic.csv', index=False)